In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold;
USE gold;


In [0]:
%sql
-- Criando tabela gold.ft_vendas_consumidor_local
create or replace table gold.ft_vendas_consumidor_local
using delta as 
select
    p.id_pedido as id_pedido,
    p.id_consumidor as id_consumidor,
    CAST(p.valor_total_pago_brl as decimal(12,2)) as valor_total_pedido_brl,
    c.cidade as cidade,
    c.estado as estado,
    CAST(p.data_pedido AS DATE) as data_pedido
from silver.ft_pedido_total p
join silver.ft_consumidores c on p.id_consumidor = c.id_consumidor;

select * from gold.ft_vendas_consumidor_local limit 15;


id_pedido,id_consumidor,valor_total_pedido_brl,cidade,estado,data_pedido
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,38.71,SAO PAULO,SP,2017-10-02
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,141.46,BARREIRAS,BA,2018-07-24
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,179.12,VIANOPOLIS,GO,2018-08-08
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,72.20,SAO GONCALO DO AMARANTE,RN,2017-11-18
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,28.62,SANTO ANDRE,SP,2018-02-13
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,175.26,CONGONHINHAS,PR,2017-07-09
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,65.95,SANTA ROSA,RS,2017-04-11
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,75.16,NILOPOLIS,RJ,2017-05-16
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,35.95,FAXINALZINHO,RS,2017-01-23
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,169.76,SOROCABA,SP,2017-07-29


In [0]:
%sql
-- View de total de compras por localidade
create or replace view gold.view_total_compras_por_consumidor as
select
    cidade,estado,
    count(distinct id_pedido) as quantidade_vendas,
    sum(valor_total_pedido_brl) as valor_total_localidade
from gold.ft_vendas_consumidor_local
group by cidade, estado;

select * from gold.view_total_compras_por_consumidor order by valor_total_localidade desc limit 20;



cidade,estado,quantidade_vendas,valor_total_localidade
SAO PAULO,SP,15540,2203373.09
RIO DE JANEIRO,RJ,6882,1161927.36
BELO HORIZONTE,MG,2773,421765.12
BRASILIA,DF,2131,354216.78
CURITIBA,PR,1521,247392.48
PORTO ALEGRE,RS,1379,224731.42
SALVADOR,BA,1245,218071.50
CAMPINAS,SP,1444,216248.43
GUARULHOS,SP,1189,165121.99
NITEROI,RJ,849,139996.99


In [0]:
%sql
-- Total de vendas por estado (em BRL)
select
    estado,
    sum(valor_total_pedido_brl) as valor_total_estado_brl
from gold.ft_vendas_consumidor_local
group by estado
order by valor_total_estado_brl desc;


estado,valor_total_estado_brl
SP,5998226.96
RJ,2144379.69
MG,1872257.26
RS,890898.54
PR,811156.38
SC,623086.43
BA,616645.82
DF,355141.08
GO,350092.31
ES,325967.55


**Parte 2:
**

In [0]:
%sql
-- Tabela de atrasos por pedido/local/vendedor
create or replace table gold.ft_atrasos_pedidos_local_vendedor
using delta as
select
  p.id_pedido as id_pedido,
  i.id_vendedor as id_vendedor,
  p.id_consumidor as id_consumidor,
  p.entrega_no_prazo as entrega_no_prazo,
  cast(p.tempo_entrega_dias as int) as tempo_entrega_dias,
  cast(p.tempo_entrega_estimado_dias as int) as tempo_entrega_estimado_dias,
  c.cidade as cidade,
  c.estado as estado
from silver.ft_pedidos       p
join silver.ft_consumidores  c on p.id_consumidor = c.id_consumidor
join silver.ft_itens_pedidos i on p.id_pedido = i.id_pedido;

select * from gold.ft_atrasos_pedidos_local_vendedor limit 15;


id_pedido,id_vendedor,id_consumidor,entrega_no_prazo,tempo_entrega_dias,tempo_entrega_estimado_dias,cidade,estado
ccbabeb0b02433bd0fcbac46e70339f2,16090f2ca825584b5a147ab24aa30c86,c77ee2d8ba1614a4d489a44166894938,SIM,18,22,UBERABA,MG
c6bf92017bd40729c135b58b643f64c2,36a968b544695394e4e9d7572688598f,3d3c463710ea6e8dd9a63c1110eeb06b,SIM,7,8,SUMARE,SP
ab87dc5a5f1856a10640d5f42e4c2fd9,b410bdd36d5db7a65dcd42b7ead933b8,538a4d02876412846b966a3c057395e5,SIM,7,43,UBERABA,MG
06ff862a85c2402aa52dc9edf150bf30,7e3f87d16fb353f408d467e74fbd8014,0a978c825ff7d013133ddc7f77566172,SIM,28,28,FORTALEZA,CE
f23155f5fa9b826631c5b8e038b38393,74c7dec0a384d8a05950e629bd23bde9,21a99191298d34fb6dd0b088e821591c,SIM,12,23,RIO DE JANEIRO,RJ
69fd81b0cd556f5da5000c1ed874ed19,080102cd0a76b09e0dcf55fcacc60e05,6ad71323c11ba8a83737ccc3ea31fbc3,SIM,3,24,SAO BENTO DO SUL,SC
d40dd8018a5302969efb31bd21744cab,8e6cc767478edae941d9bd9eb778d77a,0470c47f1dd7a91d0f3b8a420589e0f7,SIM,14,21,SETE LAGOAS,MG
42560dfc8d7863a190293678f01f6bbd,f181738b150df1f37cb0bd72e705b193,0f3a81be69f12da7e2979fd1833e923d,SIM,2,15,SAO PAULO,SP
3f003568147c785083d014edfba38c48,08d0949a9a17c027262db1f3c450c26c,72d90899884781ae2fc19e49cc102fc0,SIM,14,29,FLORIANOPOLIS,SC
db192ddb0ea5a4d7a0506e7d7ec92343,7142540dd4c91e2237acb7e911c4eba2,c21e33cbee939409efff8c2ef959a275,SIM,6,20,URUPES,SP


In [0]:
%sql
-- View de tempo médio de entrega por cidade/estado
create or replace view gold.view_tempo_medio_entrega_localidade as
select
  cidade,
  estado,
  avg(tempo_entrega_dias) as tempo_medio_entrega,
  avg(tempo_entrega_estimado_dias) as tempo_medio_estimado,
    case
      when avg(tempo_entrega_dias) > avg(tempo_entrega_estimado_dias)
      Then 'SIM'
      else 'NÃO'
    end as entrega_maior_que_estimado
from gold.ft_atrasos_pedidos_local_vendedor
where entrega_no_prazo <> 'NÃO ENTREGUE'
group by cidade, estado;

-- Top 10 localidades com maior tempo médio de entrega
select *
from gold.view_tempo_medio_entrega_localidade
order by tempo_medio_entrega desc
limit 10;



cidade,estado,tempo_medio_entrega,tempo_medio_estimado,entrega_maior_que_estimado
NOVO BRASIL,ES,148.0,22.0,SIM
CAPINZAL DO NORTE,MA,109.0,31.0,SIM
ADHEMAR DE BARROS,PR,98.0,25.0,SIM
SANTA CRUZ DE GOIAS,GO,86.66666666666667,51.333333333333336,SIM
ARACE,ES,86.5,33.5,SIM
SANTA MARIA,RJ,84.0,28.0,SIM
MARCELINO VIEIRA,RN,78.0,29.0,SIM
SERRA DOS DOURADOS,PR,75.0,23.0,SIM
ARAMARI,BA,74.0,31.0,SIM
HUMILDES,BA,72.0,30.0,SIM


In [0]:
%sql
-- Top 10 das cidades mais eficientes (entregam antes do esperado)

select
  cidade,
  estado,
  tempo_medio_entrega,
  tempo_medio_estimado,
  (tempo_medio_estimado - tempo_medio_entrega) as folga_media_dias
from gold.view_tempo_medio_entrega_localidade
where entrega_maior_que_estimado = 'NÃO'
order by folga_media_dias desc
limit 10;


cidade,estado,tempo_medio_entrega,tempo_medio_estimado,folga_media_dias
JURUTI,PA,28.0,88.0,60.0
FATIMA,TO,14.0,71.0,57.0
PORTALEGRE,RN,40.666666666666664,85.0,44.333333333333336
SANTA TEREZINHA,PE,17.0,61.0,44.0
APUAREMA,BA,23.0,65.0,42.0
ALVORADA D'OESTE,RO,41.0,82.0,41.0
XEXEU,PE,9.0,50.0,41.0
JAPOATA,SE,26.0,67.0,41.0
ACARI,RN,13.0,53.0,40.0
BONFIM,RR,18.0,58.0,40.0


In [0]:
%sql
-- View de pontualidade por vendedor
create or replace view gold.view_vendedor_pontualidade as
select
  id_vendedor,
  count(distinct id_pedido) as total_pedidos,
  sum(case
        when entrega_no_prazo in ('NÃO', 'NÃO ENTREGUE') 
        then 1
        else 0
      end ) as total_atrasados,
    100.0 * sum( case
                  when entrega_no_prazo in ('NÃO', 'NÃO ENTREGUE')
                  then 1
                  else 0
                end) / count(distinct id_pedido) as percentual_atraso
from gold.ft_atrasos_pedidos_local_vendedor
group by id_vendedor;

-- Vendedores com maior percentual de atraso
select
  id_vendedor,
  total_pedidos,
  total_atrasados,
  percentual_atraso
from gold.view_vendedor_pontualidade
where total_pedidos >= 15
order by percentual_atraso desc
limit 20;


id_vendedor,total_pedidos,total_atrasados,percentual_atraso
4342d4b2ba6b161468c63a7e7cfce593,20,19,95.00000000000000
2709af9587499e95e803a6498a5a56e9,26,24,92.30769230769231
b1b3948701c5c72445495bd161b83a4c,18,13,72.22222222222222
adcf50477d6a1f5d91bb9d54c4903cb5,19,12,63.15789473684211
ffff564a4f9085cd26170f4732393726,20,12,60.00000000000000
cb5df0dcb9e280b1780e9d589889f2c7,16,9,56.25000000000000
ec4608a1f76453166bb312b2968aeaf4,18,10,55.55555555555556
ecccfa2bb93b34a3bf033cc5d1dcdc69,15,7,46.66666666666667
b19f3ca2ea475913750f25a5c37c8d8f,24,11,45.83333333333333
821fb029fc6e495ca4f08a35d51e53a5,28,12,42.85714285714286


**Parte 3:**

In [0]:
%sql
-- Dimensão de Tempo (gold.dm_tempo)
create or replace table gold.dm_tempo
using delta as
with limites as (
  select
    min(data_pedido) as data_min,
    max(data_pedido) as data_max
    from silver.ft_pedido_total
),
datas as (
  select explode(sequence(data_min, data_max, interval 1 day)) as ch_tempo
  from limites
)
select
    ch_tempo as tempo,
    year(tempo) as ano,
    quarter(tempo) as trimestre,
    month(tempo) as mes,
    weekofyear(tempo) as semana_do_ano,
    day(tempo) as dia,
    dayofweek(tempo) as dia_da_semana_num,
    
    case dayofweek(tempo)
      when 1 then 'Domingo'
      when 2 then 'Segunda-feira'
      when 3 then 'Terça-feira'
      when 4 then 'Quarta-feira'
      when 5 then 'Quinta-feira'
      when 6 then 'Sexta-feira'
      when 7 then 'Sábado'
    end as dia_da_semana_nome,

    case month(tempo)
      when 1  then 'Janeiro'
      when 2  then 'Fevereiro'
      when 3  then 'Março'
      when 4  then 'Abril'
      when 5  then 'Maio'
      when 6  then 'Junho'
      when 7  then 'Julho'
      when 8  then 'Agosto'
      when 9  then 'Setembro'
      when 10 then 'Outubro'
      when 11 then 'Novembro'
      when 12 then 'Dezembro'
    end as mes_nome,
    
    case
      when dayofweek(tempo) in (1, 7) then 'Sim'
      else 'Não'
    end as fim_de_semana
from datas;

select * from gold.dm_tempo order by tempo limit 25;


tempo,ano,trimestre,mes,semana_do_ano,dia,dia_da_semana_num,dia_da_semana_nome,mes_nome,fim_de_semana
2016-09-04,2016,3,9,35,4,1,Domingo,Setembro,Sim
2016-09-05,2016,3,9,36,5,2,Segunda-feira,Setembro,Não
2016-09-06,2016,3,9,36,6,3,Terça-feira,Setembro,Não
2016-09-07,2016,3,9,36,7,4,Quarta-feira,Setembro,Não
2016-09-08,2016,3,9,36,8,5,Quinta-feira,Setembro,Não
2016-09-09,2016,3,9,36,9,6,Sexta-feira,Setembro,Não
2016-09-10,2016,3,9,36,10,7,Sábado,Setembro,Sim
2016-09-11,2016,3,9,36,11,1,Domingo,Setembro,Sim
2016-09-12,2016,3,9,37,12,2,Segunda-feira,Setembro,Não
2016-09-13,2016,3,9,37,13,3,Terça-feira,Setembro,Não


In [0]:
%sql
-- Fato de vendas geral (pedido + item + tempo + avaliação + dólar)
create or replace table gold.ft_vendas_geral
using delta as
with avaliacoes_por_pedido as (
  select
    id_pedido,
    avg(avaliacao) as avaliacao_pedido
    from silver.ft_avaliacoes_pedidos
    group by id_pedido
),
cotacao_por_data as (
  select
    data as data_cotacao, cotacao_dolar
    from silver.dm_cotacao_dolar
)
select
  i.id_pedido as id_pedido,
  i.id_item as id_item,
  pt.id_consumidor as fk_cliente,
  i.id_produto as fk_produto,
  i.id_vendedor as fk_vendedor,
  pt.data_pedido as fk_tempo,
  pt.status as status_pedido,
  p.tempo_entrega_dias as tempo_entrega_dias,
  p.entrega_no_prazo as entrega_no_prazo,
  cast(i.`preço_brl` as decimal(12,2)) as valor_produto_brl,
  cast(i.`preço_frete` as decimal(12,2)) as valor_frete_brl,
  cast(i.`preço_brl` + i.`preço_frete` as decimal(12,2)) as valor_total_item_brl,
  cast(i.`preço_brl` / c.cotacao_dolar as decimal (12,2)) as valor_produto_usd,
  cast(i.`preço_frete` / c.cotacao_dolar as decimal (12,2)) as valor_frete_usd,
  cast( (i.`preço_brl` + i.`preço_frete`) / c.cotacao_dolar as decimal (12,2)) as valor_total_item_usd,
  cast(c.cotacao_dolar as decimal (8,4)) as cotacao_dolar,
  cast(a.avaliacao_pedido as decimal (3,2)) as avaliacao_pedido

from silver.ft_itens_pedidos i
join silver.ft_pedido_total pt on i.id_pedido = pt.id_pedido
join silver.ft_pedidos p on i.id_pedido = p.id_pedido
left join avaliacoes_por_pedido a on i.id_pedido = a.id_pedido
left join cotacao_por_data c on pt.data_pedido = c.data_cotacao;

select * from gold.ft_vendas_geral limit 15;


id_pedido,id_item,fk_cliente,fk_produto,fk_vendedor,fk_tempo,status_pedido,tempo_entrega_dias,entrega_no_prazo,valor_produto_brl,valor_frete_brl,valor_total_item_brl,valor_produto_usd,valor_frete_usd,valor_total_item_usd,cotacao_dolar,avaliacao_pedido
019886de8f385a39b75bedbb726fd4ef,1,8cf88d7ba142365ef2ca619ef06f9a0f,e9a69340883a438c3f91739d14d3a56d,1b4c3a6f53068f0b6944d2d005c9fc89,2018-02-10,entregue,13,SIM,159.90,28.50,188.40,null,null,null,null,5.00
06ff862a85c2402aa52dc9edf150bf30,1,0a978c825ff7d013133ddc7f77566172,4ce9ab528124f89e091b17d11aa2e97c,7e3f87d16fb353f408d467e74fbd8014,2017-11-30,entregue,28,SIM,41.90,17.63,59.53,null,null,null,null,3.00
110ac0768c3e3a78e2937b1cb1ea3395,1,770d7945efb2aa13cbcf28eb7a353def,1b7ce992a80ac036dd9ab73d08289712,582d4f8675b945722eda7c0cb61ba4c7,2017-06-09,entregue,5,SIM,25.50,15.10,40.60,null,null,null,null,5.00
12dd6fe47b1a7c9e5742662223880dcc,1,95a911adbd9c6dee5b6ba9f764678fa3,63f4d40c05db6ade462cecef857eec34,fec6275253471ace26d209bbaa64cd0f,2018-01-24,entregue,14,SIM,284.00,39.54,323.54,null,null,null,null,5.00
139be8870b91e71fd70bb366305c8cde,1,7f0f4bcffd7d9085eed3ad0a7814dd33,4c1e109ecdf58453de365d217cefa64c,4e922959ae960d389249c378d1c939f5,2017-05-13,entregue,9,SIM,120.00,13.18,133.18,null,null,null,null,5.00
1573c1514d9e6b02151d3f7a1e9b0b20,1,223811908c4b45b0238ea546789c3c4b,a87567bdce60de6f901e3d7c6d8df820,318f287a62ab7ac10b703ac37435a231,2018-02-15,entregue,6,SIM,74.00,13.08,87.08,null,null,null,null,4.00
1573c1514d9e6b02151d3f7a1e9b0b20,2,223811908c4b45b0238ea546789c3c4b,a87567bdce60de6f901e3d7c6d8df820,318f287a62ab7ac10b703ac37435a231,2018-02-15,entregue,6,SIM,74.00,13.08,87.08,null,null,null,null,4.00
17510720f670481600a76f4d188d7b5b,1,a50b97c5344a35fdbc10c442e88e720d,7822d1f226e657d5c98ce5a18bd0ad94,582d4f8675b945722eda7c0cb61ba4c7,2018-01-14,entregue,9,SIM,14.99,15.10,30.09,null,null,null,null,1.00
1816598bbdabc09eaf2174d9e02e0606,1,e9024de0cde5de0becbf917e1ea5a17f,37be426992f83a0fe7922c694f0d6947,7ecd59e5e20407131822c1a68ac59c1f,2018-01-14,entregue,23,SIM,69.99,27.03,97.02,null,null,null,null,4.00
1e34943faee67957ebdc93ece5a8f906,1,e7d9c6487ba15bee967f0f1305c75c8b,41cd333a4af5e36df81fc0b83ccf8e2d,5dceca129747e92ff8ef7a997dc4f8ca,2018-03-26,entregue,8,SIM,388.00,31.03,419.03,null,null,null,null,4.00


In [0]:
%sql
create or replace view gold.view_vendas_por_periodo as
select
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,
    COUNT(DISTINCT f.id_pedido) as total_pedidos,
    COUNT(*) as total_itens,
    SUM(f.valor_total_item_brl) as receita_total_brl,
    SUM(f.valor_total_item_usd) as receita_total_usd,
    AVG(f.valor_total_item_brl) as ticket_medio_brl,
    AVG(f.avaliacao_pedido) as avaliacao_media
from gold.ft_vendas_geral f
join gold.dm_tempo t on f.fk_tempo = t.tempo
group by
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num;

select * from gold.view_vendas_por_periodo order by ano, mes, dia limit 20;

ano,trimestre,mes,mes_nome,dia,dia_da_semana_num,total_pedidos,total_itens,receita_total_brl,receita_total_usd,ticket_medio_brl,avaliacao_media
2016,3,9,Setembro,4,1,1,2,136.23,null,68.115000,1.000000
2016,3,9,Setembro,5,2,1,1,75.06,null,75.060000,1.000000
2016,3,9,Setembro,15,5,1,3,143.46,null,47.820000,1.000000
2016,4,10,Outubro,2,1,1,1,109.34,null,109.340000,1.000000
2016,4,10,Outubro,3,2,8,8,595.14,null,74.392500,3.666667
2016,4,10,Outubro,4,3,60,69,11295.48,null,163.702609,3.806452
2016,4,10,Outubro,5,4,42,56,9645.94,null,172.248929,3.395833
2016,4,10,Outubro,6,5,49,58,9131.23,null,157.435000,3.454545
2016,4,10,Outubro,7,6,45,52,8135.97,null,156.460962,3.377778
2016,4,10,Outubro,8,7,40,45,9383.37,null,208.519333,4.102564


In [0]:
%sql
-- Dia da semana com maior receita total em BRL
select
    dia_da_semana_num,
    case dia_da_semana_num
        when 1 then 'Domingo'
        when 2 then 'Segunda-feira'
        when 3 then 'Terça-feira'
        when 4 then 'Quarta-feira'
        when 5 then 'Quinta-feira'
        when 6 then 'Sexta-feira'
        when 7 then 'Sábado'
    end as dia_da_semana_nome,
    sum(receita_total_brl) AS receita_total_brl
from gold.view_vendas_por_periodo
Group by dia_da_semana_num, dia_da_semana_nome
order by receita_total_brl desc
limit 1;


dia_da_semana_num,dia_da_semana_nome,receita_total_brl
2,Segunda-feira,2600533.82


In [0]:
%sql
WITH ultimo_ano AS (
    SELECT MAX(ano) AS ano
    FROM gold.view_vendas_por_periodo
)
SELECT
    v.ano,
    v.mes,
    v.mes_nome,
    AVG(v.ticket_medio_brl) AS ticket_medio_brl
FROM gold.view_vendas_por_periodo v
JOIN ultimo_ano u
    ON v.ano = u.ano
GROUP BY v.ano, v.mes, v.mes_nome
ORDER BY ticket_medio_brl DESC
LIMIT 1;


ano,mes,mes_nome,ticket_medio_brl
2018,9,Setembro,166.4600000000


In [0]:
%sql
create or replace view gold.view_top_produto as
select
    f.fk_produto as id_produto,
    p.categoria_produto as categoria_produto,
    COUNT(*) as quantidade_vendida,
    COUNT(DISTINCT f.id_pedido) as total_pedidos,
    SUM(f.valor_total_item_brl) as receita_brl,
    SUM(f.valor_total_item_usd) as receita_usd,
    AVG(f.valor_produto_brl) as preco_medio_brl,
    AVG(f.avaliacao_pedido) as avaliacao_media,
    AVG(p.peso_produto_g) as peso_medio_gramas
from gold.ft_vendas_geral f
join silver.ft_produtos p on f.fk_produto = p.id_produto
group by f.fk_produto, p.categoria_produto;

select *
from gold.view_top_produto
order by receita_brl desc
limit 15;


id_produto,categoria_produto,quantidade_vendida,total_pedidos,receita_brl,receita_usd,preco_medio_brl,avaliacao_media,peso_medio_gramas
bb50f2e236e5eea0100680137654686c,beleza_saude,195,187,67606.10,null,327.615385,4.218750,400.0
d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,343,323,60976.03,null,137.651633,4.223926,6550.0
6cdd53843498f92890544667809f1595,beleza_saude,156,151,59093.99,null,350.834615,4.307190,900.0
99a4788cb24856965c36a24e339b6058,cama_mesa_banho,488,467,51071.60,null,88.167131,3.904301,1383.0
d6160fb7873f184099d9bc95e30376af,pcs,35,35,50326.18,null,1397.124000,4.588235,2600.0
3dd2a17168ec895c781a9191c1e95ad7,informatica_acessorios,274,255,48212.22,null,149.936496,4.202290,533.0
aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,527,431,44820.76,null,71.364137,4.036072,2600.0
5f504b3a1c75b73d6151be81eb05bdc9,cool_stuff,63,63,41725.81,null,598.950794,4.548387,15350.0
25c38557cf793876c5abdd5931f922db,bebes,38,38,40311.95,null,1023.876842,2.771429,15800.0
53b36df67ebb7c41585e8d54d6772e08,relogios_presentes,323,306,39957.93,null,116.666935,4.198083,584.0


In [0]:
%sql
create or replace view gold.view_vendas_produtos_esteticos as
with fashion_vendas as (
    select
        t.ano,
        t.mes,
        p.categoria_produto,
        f.id_pedido,
        f.id_item,
        f.valor_total_item_brl,
        f.valor_total_item_usd,
        f.avaliacao_pedido
    from gold.ft_vendas_geral f
    join gold.dm_tempo t on f.fk_tempo = t.tempo
    join silver.ft_produtos p on f.fk_produto = p.id_produto
    where p.categoria_produto like 'fashion%'
)

select
    ano,
    mes,
    categoria_produto,
    COUNT(DISTINCT id_pedido) as total_pedidos,
    COUNT(*) as total_itens_vendidos,
    SUM(valor_total_item_brl) as receita_total_brl,
    SUM(valor_total_item_usd) as receita_total_usd,
    AVG(valor_total_item_brl) as ticket_medio_brl,
    AVG(valor_total_item_usd) as ticket_medio_usd,
    AVG(avaliacao_pedido) as avaliacao_media
from fashion_vendas
group by ano, mes, categoria_produto;

select *
from gold.view_vendas_produtos_esteticos
order by ano, mes, receita_total_brl desc
limit 20;


ano,mes,categoria_produto,total_pedidos,total_itens_vendidos,receita_total_brl,receita_total_usd,ticket_medio_brl,ticket_medio_usd,avaliacao_media
2016,10,fashion_bolsas_e_acessorios,8,9,656.49,null,72.943333,null,3.222222
2016,10,fashion_roupa_feminina,1,1,69.63,null,69.630000,null,5.000000
2016,10,fashion_calcados,1,1,40.95,null,40.950000,null,5.000000
2016,10,fashion_roupa_masculina,1,1,35.86,null,35.860000,null,1.000000
2016,12,fashion_bolsas_e_acessorios,1,1,19.62,null,19.620000,null,5.000000
2017,1,fashion_bolsas_e_acessorios,30,35,2136.61,null,61.046000,null,4.451613
2017,1,fashion_underwear_e_moda_praia,2,2,171.15,null,85.575000,null,3.000000
2017,1,fashion_roupa_masculina,1,1,159.11,null,159.110000,null,5.000000
2017,1,fashion_roupa_feminina,2,2,102.81,null,51.405000,null,4.000000
2017,1,fashion_calcados,1,1,49.01,null,49.010000,null,5.000000
